In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt
from UsefulFunctions import rolling_forecast, rolling_forecast_multivariate, FitLSTM_multilayer, FitLSTM_1layer, FitLSTM_ext

# Task 2.1

In [ ]:
# Load electricity spot price data
df = pd.read_csv("Elspotprices2nd.csv")
df["HourUTC"] = pd.to_datetime(df["HourUTC"])
df.set_index("HourUTC", inplace=True)
df = df.sort_index()

# Split data into training (Jan 2019 to Aug 2024) and testing (Sep 2024)
lstm1_train_data = df.loc[:"2024-08-31"].values.reshape(-1, 1)
lstm1_test_data = df.loc["2024-09-01":"2024-09-30"].values.reshape(-1, 1)

# Normalize both training and testing data
lstm1_scaler = MinMaxScaler()
lstm1_train_scaled = lstm1_scaler.fit_transform(lstm1_train_data)
lstm1_test_scaled = lstm1_scaler.transform(lstm1_test_data)

In [ ]:
# Define input features and hyperparameters
window_size = 24
n_lookahead = 24
n_neurons = 64
n_features = 1
epochs = 10
dropout = 0

# Fit the LSTM model
# The ordered_validation parameter is set to True to ensure that the validation set is ordered in time,
# and not randomly shuffled. This is important for time series data to maintain the temporal order.
lstm2_model = FitLSTM_1layer(lstm1_train_scaled, window_size, n_features, 
                             n_lookahead, n_neurons, epochs, dropout, ordered_validation=True)
# lstm2_model.save("lstm1_model_test.keras") # Uncomment to save the model

# Generate predictions
lstm1_pred_scaled = rolling_forecast(lstm2_model, lstm1_train_scaled, lstm1_test_scaled, window_size, n_lookahead)

# Inverse transform to get the actual values from the scaled predictions
lstm1_pred_inv = lstm1_scaler.inverse_transform(lstm1_pred_scaled.reshape(-1, 1))

# Forecast with persistence (naive forecast using previous values)
Persistence = np.concatenate((lstm1_train_data[-n_lookahead:], # Start with last <n_lookahead> hours of training data
                              lstm1_test_data[:-n_lookahead]), # Add the test data to the persistence forecast
                              axis=0)  # Combine arrays

# Calculate and print RMSE values for persistence and forecasts
rmse_lstm1 = root_mean_squared_error(lstm1_test_data, lstm1_pred_inv)
rmse_persistence = root_mean_squared_error(Persistence, lstm1_test_data)
print(f"RMSE for LSTM (No exogenous): {rmse_lstm1:.2f}")
print(f"RMSE for Persistence: {rmse_persistence:.2f}")

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(lstm1_train_data) + 1), lstm1_train_data, color="black", label="Training set")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_pred_inv.flatten()) + 1), lstm1_pred_inv.flatten(), color="blue", label="Forecasted values")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Persistence) + 1), Persistence, color="green", label="Persistence")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_test_data.flatten()) + 1), lstm1_test_data.flatten(), color="red", label="Actual values")
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.xlim([len(lstm1_train_data) - 7 * 24, len(lstm1_train_data) + len(lstm1_test_data)])
plt.tight_layout()
plt.show()

### Version with multilayer LSTM
The multilayer setup leads to higher rmse.

In [ ]:
# Define input features and hyperparameters
n_features = 1  # No external input features, only the target variable
n_steps = 24  # The number of time steps the model will look back
n_lookahead = 24  # The number of steps the model will predict ahead
n_neurons = 64  # Number of neurons in the LSTM layer
n_neurons_dense = 36  # Number of neurons in the dense layer
epochs = 10  # Number of training epochs
dropout1 = 0.05  # Dropout rate for the LSTM layer
dropout2 = 0.05  # Dropout rate for the dense layer

# Fit the LSTM model
model = FitLSTM_multilayer(lstm1_train_scaled, n_steps, n_features, n_lookahead, 
                n_neurons, n_neurons_dense, epochs, dropout1, dropout2)
# model.save("The_modelv2.keras") # Uncomment to save the model

# Generate predictions
lstm1_pred_scaled = rolling_forecast(model, lstm1_train_scaled, lstm1_test_scaled, n_steps, n_lookahead)

# Inverse transform to get the actual values from the scaled predictions
Forecasts = lstm1_scaler.inverse_transform(lstm1_pred_scaled.reshape(-1, 1))

# Forecast with persistence (naive forecast using previous values)
Persistence = np.concatenate((lstm1_train_data[-n_lookahead:], # Start with last <n_lookahead> hours of training data
                              lstm1_test_data[:-n_lookahead]), # Add the test data to the persistence forecast
                              axis=0)  # Combine arrays

# Calculate and print RMSE values for persistence and forecasts
RMSE_P = root_mean_squared_error(Persistence, lstm1_test_data)
RMSE_F = root_mean_squared_error(Forecasts, lstm1_test_data)
print("RMSE for weekly persistence: ", round(RMSE_P))
print("RMSE for forecasts: ", round(RMSE_F))

# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(lstm1_train_data) + 1), lstm1_train_data, color="black", label="Training set")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Forecasts) + 1), Forecasts, color="blue", label="Forecasted values")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(Persistence) + 1), Persistence, color="green", label="Persistence")
plt.plot(np.arange(len(lstm1_train_data) + 1, len(lstm1_train_data) + len(lstm1_test_data) + 1), lstm1_test_data, color="red", label="Actual values")
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.xlim([len(lstm1_train_data) - 7 * 24, len(lstm1_train_data) + len(lstm1_test_data)])
plt.tight_layout()
plt.show()

# Task 2.2

In [ ]:
# Load price data
df_prices = pd.read_csv("Elspotprices2nd.csv")
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices.set_index("HourUTC", inplace=True)
df_prices = df_prices.sort_index()

# Load exogenous data
df_exo = pd.read_csv("ProdConData.csv")
df_exo["HourUTC"] = pd.to_datetime(df_exo["HourUTC"])
df_exo.set_index("HourUTC", inplace=True)
df_exo = df_exo.sort_index()

# Merge datasets
df_combined = df_prices.join(df_exo, how='inner')

# Select target + exogenous features
exogenous_vars = ["GrossConsumptionMWh", "OffshoreWindGe100MW_MWh", "SolarPowerGe40kW_MWh"]
lstm2_features = ["SpotPriceDKK"] + exogenous_vars
df_lstm2 = df_combined[lstm2_features].dropna()

# Train/test split
lstm2_train = df_lstm2.loc[:"2024-08-31"]
lstm2_test = df_lstm2.loc["2024-09-01":"2024-09-30"]

# Normalize
lstm2_scaler = MinMaxScaler()
lstm2_train_scaled = lstm2_scaler.fit_transform(lstm2_train)
lstm2_test_scaled = lstm2_scaler.transform(lstm2_test)

# Fit a separate scaler for the target variable to inverse transform the forecasted values back to original scale
# This ensures that the predictions we make can be transformed back to the original scale
lstm2_scaler_y = MinMaxScaler()
lstm2_scaler_y = lstm2_scaler_y.fit(lstm2_train["SpotPriceDKK"].values.reshape(-1, 1))

In [ ]:
# Define input features and hyperparameters
window_size = 24
n_lookahead = 24
n_neurons = 80
n_features = len(lstm2_features)
epochs = 10
dropout = 0.06

lstm2_model = FitLSTM_ext(lstm2_train_scaled, window_size, n_features, n_lookahead, 
                          n_neurons, epochs, dropout, ordered_validation=True)

# Forecast
lstm2_pred_scaled = rolling_forecast_multivariate(lstm2_model, lstm2_train_scaled, 
                          lstm2_test_scaled, window_size, n_lookahead, n_features)

# Inverse transform only price predictions
lstm2_pred_inv = lstm2_scaler_y.inverse_transform(lstm2_pred_scaled.reshape(-1, 1))[:,0]

# Ground truth
lstm2_true = lstm2_test["SpotPriceDKK"].values

# RMSE for the forecasted values
RMSE_F = root_mean_squared_error(lstm2_true, lstm2_pred_inv)  
print(f"LSTM (with exogenous vars) RMSE: {RMSE_F:.2f}")

In [ ]:
lstm2_train_true = lstm2_train["SpotPriceDKK"].values
# Plot the forecasts
plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(lstm2_train_true) + 1), lstm2_train_true, color="black", label="Training set")
plt.plot(np.arange(len(lstm2_train_true) + 1, len(lstm2_train_true) + len(lstm2_pred_inv.flatten()) + 1), lstm2_pred_inv.flatten(), color="blue", label="Forecasted values")
plt.plot(np.arange(len(lstm2_train_true) + 1, len(lstm2_train_true) + len(lstm2_true.flatten()) + 1), lstm2_true.flatten(), color="red", label="Actual values")
plt.legend(loc="upper left")
plt.grid(alpha=0.25)
plt.xlim([len(lstm2_train_true) - 7 * 24, len(lstm2_train_true) + len(lstm2_true)])
plt.tight_layout()
plt.show()

In [ ]:
# lstm2_model.save("lstm2_model_good3.keras")